In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re
dict_file = '../data/survey/7041-1996-2015_City_Survey_DataSF.xlsx'
data_dict = pd.read_excel(dict_file, 0, skiprows=3)
data_dict['Variable'] = data_dict['Variable'].str.replace('\n', '') 
data_dict = data_dict.set_index('Variable')

#keep only questions which have FDCBA grades
data_dict1 = data_dict.ix[['id', 'year', 'finweigh', 'zipcode', 'district', 'region']]
data_dict2 = data_dict[data_dict['Value Labels'].str.contains('F-Failing', flags=re.IGNORECASE).fillna(False)]
# remove column 'swcndnbd' as it's not present in data csv
data_dict2 = data_dict2.drop('swcndnbd')
data_dict = pd.concat([data_dict1, data_dict2])

In [ ]:
survey_df = pd.read_csv('../data/survey/San_Francisco_City_Survey_Data_1996-2015.csv')

# keep only valid cols
survey_df = survey_df[list(set(data_dict.index) & set(survey_df.columns))]

# remove values of 6 and 7 (dont know, not provided responses)
replace_dict = {col: {6: np.nan, 7: np.nan} for col in data_dict2.index}
survey_df = survey_df.replace(replace_dict)

# consider data only from 2009 - 2015
survey_df = survey_df[survey_df['year'].isin([2009, 2011, 2013, 2015])]

# consider only 941XX zip codes for SF city
survey_df = survey_df[(survey_df['zipcode'] > 94100) & (survey_df['zipcode'] < 94200)]
print survey_df.shape

In [ ]:
cat_qs_counts = data_dict2['Category'].value_counts()
cat_qs_counts

In [ ]:
## weight for each category is the count of the number of questions
# weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]

# weight_mat = survey_df[data_dict2.index].notnull() * weights
# satisfaction_scores = (weight_mat * survey_df[data_dict2.index]).sum(axis=1) / weight_mat.sum(axis=1)
# scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [ ]:
## weight for each category is the count of the number of questions
weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]

# weight_mat = survey_df[data_dict2.index].notnull() / weights
satisfaction_scores = (survey_df[data_dict2.index] / weights).sum(axis=1) 
scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [ ]:
scores_by_zip.sort_values()

In [ ]:
scores_by_zip.index = scores_by_zip.index.map(lambda x: int(x))
scores_by_zip

In [ ]:
# scores_by_zip.to_csv('/Users/chakri/bayeshack/analysis_data/scores_by_zipcodes.csv')